In [15]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [16]:
data = pd.read_csv('random10ksample.csv', sep=';')
print(data.head())

   MovieID  CustomerID  Rate
0     5054     1197338     5
1     2218     1414981     5
2     2443     2628834     2
3    15409      740628     5
4    14830     2096735     4


Embeding


In [17]:

user_id_map = {user_id: idx for idx, user_id in enumerate(data['CustomerID'].unique())}
movie_id_map = {movie_id: idx for idx, movie_id in enumerate(data['MovieID'].unique())}

data['user'] = data['CustomerID'].map(user_id_map)
data['movie'] = data['MovieID'].map(movie_id_map)

Train-test split


In [18]:
X = data[['user', 'movie']].values
y = data['Rate'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Params


In [19]:
num_users = len(user_id_map)
num_movies = len(movie_id_map)
embedding_size = 50

Neural Collaborative Filtering Model

1-User and movie embedding:

Here, users and movies are represented in a vector dimension of a certain size. Then, the embedding vector for each user or movie is made suitable as input to dense layers (in short, flat). Dense layers work with one-dimensional arrays. Flattening is a must, the model cannot send this data to dense layers.

In [20]:
# User and movie embedding
user_input = tf.keras.layers.Input(shape=(1,), name="user_input")
movie_input = tf.keras.layers.Input(shape=(1,), name="movie_input")

user_embedding = tf.keras.layers.Embedding(input_dim=num_users, output_dim=embedding_size, name="user_embedding")(user_input)
movie_embedding = tf.keras.layers.Embedding(input_dim=num_movies, output_dim=embedding_size, name="movie_embedding")(movie_input)

# Flatten
user_flatten = tf.keras.layers.Flatten()(user_embedding)
movie_flatten = tf.keras.layers.Flatten()(movie_embedding)

2-) Concatenation: Here, the user and film vectors prepared for the dense layers are combined.

In [21]:
concat = tf.keras.layers.Concatenate()([user_flatten, movie_flatten])

3-)Fully connected layers are created here. In the first layer, the number of neurons is 128. In other words, 128 different features are tried to be learned. The ReLu activation function brings the negatives closer to zero and leaves the positives as they are. Non-linear relationships are learned. Concat is the combination of the user and movie vectors we created above. Our first layer starts the learning process from here. In the 2nd layer; it adds 64 more features to the 128 features learned in the first layer. The dense1 at the end shows that it uses the features of the previous layer. In the 3rd layer, the learning process is completed by taking the output of the 2nd layer.

In [22]:
dense1 = tf.keras.layers.Dense(64, activation='relu', name="dense1")(concat)
dense2 = tf.keras.layers.Dense(32, activation='relu', name="dense2")(dense1)
dense3 = tf.keras.layers.Dense(32, activation='relu', name="dense3")(dense2)


Outpu layer.

In [23]:
output = tf.keras.layers.Dense(1, activation='linear', name="output")(dense3)

Here the model is created. The inputs are the user and movie inputs, which are the information the model will receive, and the output is the user's predicted score to be predicted. Adam optimization algorithm was used. The model's error was measured with MSE. (RMSE will be calculated by taking the square of all values ​​while calculating the overall accuracy.)


In [24]:
model = tf.keras.Model(inputs=[user_input, movie_input], outputs=output)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ user_input (InputLayer)   │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ movie_input (InputLayer)  │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ user_embedding            │ (None, 1, 50)          │        485,050 │ user_input[0][0]       │
│ (Embedding)               │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ movie_embedding           │ (None, 1, 50)          │        174,500 │ movie_input[0][0]      │
│ (Embedding)               │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_2 (Flatten)       │ (None, 50)             │              0 │ user_embedding[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_3 (Flatten)       │ (None, 50)             │              0 │ movie_embedding[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_1             │ (None, 100)            │              0 │ flatten_2[0][0],       │
│ (Concatenate)             │                        │                │ flatten_3[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense1 (Dense)            │ (None, 64)             │          6,464 │ concatenate_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense2 (Dense)            │ (None, 32)             │          2,080 │ dense1[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense3 (Dense)            │ (None, 32)             │          1,056 │ dense2[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ output (Dense)            │ (None, 1)              │             33 │ dense3[0][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 669,183 (2.55 MB)

 Trainable params: 669,183 (2.55 MB)

 Non-trainable params: 0 (0.00 B)

The training of the model starts here. x_train is our training set. [X_train[:, 0], X_train[:, 1]] Here the movie and user columns are separated from each other. These two columns are the input data from which the model will receive user and movie information. y_train contains the target data that the model will try to predict. In this case, the target value will be the score that the user will give to the movie. validation_data is the validation step. The performance of the model is measured with data that it has not seen before in the part where it is still learning. Epoch is the number of times the model makes a full transformation on the training data set. Since 10 epochs are specified here, the model will work on the training data 10 times. Batch size is actually the number of iterations. In other words, it is the number of data samples to be presented to the model. verbose is how much detailed information will be given in the output of the training process.

In [25]:
history = model.fit(
    [X_train[:, 0], X_train[:, 1]], y_train,
    validation_data=([X_test[:, 0], X_test[:, 1]], y_test),
    epochs=10, batch_size=64, verbose=1
)

Epoch 1/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - loss: 9.1108 - mse: 9.1108 - val_loss: 1.1483 - val_mse: 1.1483
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 0.8449 - mse: 0.8449 - val_loss: 1.1975 - val_mse: 1.1975
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.2438 - mse: 0.2438 - val_loss: 1.2499 - val_mse: 1.2499
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.1457 - mse: 0.1457 - val_loss: 1.2729 - val_mse: 1.2729
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.1058 - mse: 0.1058 - val_loss: 1.2645 - val_mse: 1.2645
Epoch 6/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0688 - mse: 0.0688 - val_loss: 1.2718 - val_mse: 1.2718
Epoch 7/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0475 - mse: 0.0475 - val_loss: 1.2665 - val_mse: 1.2665
Epoch 8/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0359 - mse: 0.0359 - val_loss: 1.2486 - val_mse: 1.2486
Epoch 9/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step -

# Model Prediction and Evaluation (RMSE)

In [26]:
y_pred = model.predict([X_test[:, 0], X_test[:, 1]])
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Root Mean Squared Error (RMSE): {rmse}")


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Root Mean Squared Error (RMSE): 1.1139038209299632
